# FashionMNIST Challenge 之 數據轉儲TFRecord

## 一、編程實現
### 1、將數據集劃分成為訓練集，驗證集、測試集並存儲為TFRecord文件。

In [ ]:
import os
import gzip
import numpy as np
import tensorflow as tf

def load_mnist(path='./data/', kind='train'):
    """
    加載mnist數據集
    :param path: 傳入路徑
    :param kind: 類別(train or t10k)
    :return: images 圖片數據nparrge     lables 標籤列表npaage
    """
    labels_path = os.path.join(path, '{0}-labels-idx1-ubyte.gz'.format(kind))
    images_path = os.path.join(path, '{0}-images-idx3-ubyte.gz'.format(kind))

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8, offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8, offset=16).reshape(len(labels), 784)

    return images, labels


# 發現 tensorflow 自帶了讀取 mnist格式的模塊,直接調用好啦٩(｡・ω・｡)و
from tensorflow.examples.tutorials.mnist import input_data

data = input_data.read_data_sets('./data/', validation_size=5000)


In [ ]:
def write_tfrecord(lable, data):
    """
    寫入tfrecords文件數據
    :param lable(string): 文件名
    :param data(DataSet): 數據列表
    """
    writer = tf.python_io.TFRecordWriter(lable + ".tfrecords")
    num = len(data.labels)
    for i in range(num):
        if (i + 1) % 10000 == 0:
            print("以處理{0}數據集{1}張".format(lable, i + 1))
        img = data.images[i]
        example = tf.train.Example(features=tf.train.Features(feature={
            "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[data.labels[i]])),
            'img_val': tf.train.Feature(float_list=tf.train.FloatList(value=[img][0]))
        }))  # example對象 對label及img_val 進行封裝
        writer.write(example.SerializeToString())  # 序列化字符串?
        i += 1
    print("{0}數據集處理完成".format(lable))
    writer.close()


In [ ]:
# 調用函數，寫入數據
write_tfrecord("train", data.train)
write_tfrecord("validation", data.validation)
write_tfrecord("test", data.test)

### 2、利用matplotlib等工具對TFRecord中的樣本數據進行可視化，以驗證存儲在TFRecord文件中的樣本與標記的完整性與其對應性，並對數據集有個直觀的認識。

In [ ]:
# 將製作好的 tfrecord 數據集文件讀取出來,並轉換成圖片,以驗證數據是否準確無誤
from PIL import Image

filename_queue = tf.train.string_input_producer(["train.tfrecords"])  # 讀入數據流
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)  # 返回文件名和文件
features = tf.parse_single_example(serialized_example,
                                   features={
                                       'label': tf.FixedLenFeature([], tf.int64),
                                       'img_val': tf.FixedLenFeature([28, 28], tf.float32),
                                   })  # 取出包含image和label的feature对象
image = tf.cast(features['img_val'], tf.float64)
label = tf.cast(features['label'], tf.int32)


import matplotlib.pyplot as plt
% matplotlib inline


with tf.Session() as sess:  # 開始一個對話
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for i in range(5):
        example, l = sess.run([image, label])  # 在會話中取出image和label數據
        example *= 255
        img = Image.fromarray(example)
        plt.title("The picture number is " + str(l))
        plt.figure(figsize=(5,5))
        plt.imshow(img)
        plt.show(img)
        # 以bmp格式存儲本地
        # example = np.array(example, dtype='uint8')
        # img=Image.fromarray(example)
        # img.save(str(i)+'_''Label_'+str(l)+'.bmp','bmp')# 保存圖
    coord.request_stop()
    coord.join(threads)
    

<img src="./img/srecc.png">

## 二、實現總結
1. 對數據成功完成數據轉儲並再現了數據，數據準確無誤
2. 嗯，沒什麼說的了。

#### 完成時間：2017年11月15日，完成人：土豆豆
